In [149]:
# change to store dataset
STORE_PUB = True

# change for different location

# locations: ravensburg, mannheim, heidenheim, karlsruhe, campus-horb, stuttgart
# locations: heilbronn, loerrach, mosbach, villingen-schwenningen

location = "mosbach"

In [150]:
import math
import re
from collections import Counter

import datetime
import pandas as pd
import ast

In [151]:
WORD = re.compile(r"\w+")

# get current year to save in applicable folder
current_year = datetime.date.today().year

In [152]:
### data import ###

J_RATINGS_SCIMAGO = pd.read_csv(f'../data/{current_year}/journal_ratings_scimago.csv', sep=";")
J_RATINGS_VHB = pd.read_csv(f'../data/{current_year}/journal_ratings_vhb.csv')

publications_gs = pd.read_csv(f'../data/{current_year}/publications_{location}_gs.csv')
publications_rg = pd.read_csv(f'../data/{current_year}/publications_{location}_rg.csv')

In [153]:
# functions to get similarity of strings using cosine similarity

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)


    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [154]:
def is_similar(text1, text2):
    
    vector1 = text_to_vector(str(text1).lower())
    vector2 = text_to_vector(str(text2).lower())

    cosine = get_cosine(vector1, vector2)
    
    if cosine > 0.6:
        similar = True
    else:
        similar = False
        
    return similar

In [155]:
def get_not_none_string(string1, string2):
    
    if string1 and not string2:
        return string1
    
    elif string2 and not string1:
        return string2
    
    elif string1 and string2:
        return max([string1, string2], key=len)
    
    else:
        return None

In [156]:
def get_ratings(publisher):
    
    publisher = str(publisher)

    if publisher == "nan":
        rating_vhb = None
        issn_vhb = None
        rating_sci = None
        issn_sci = None
        
    else:
        
        rating_database = J_RATINGS_VHB
        if publisher in rating_database['JOURNAL']:
            rating_vhb = rating_database.loc[rating_database['JOURNAL'] == publisher, 'JQ3'].to_numpy()[0]
            issn_vhb = rating_database.loc[rating_database['JOURNAL'] == publisher, 'ISSN'].to_numpy()[0]

        elif index_similar := [i for i, x in enumerate(rating_database['JOURNAL']) if is_similar(publisher,x)]:
            rating_vhb = rating_database.loc[index_similar[0], 'JQ3']
            issn_vhb = rating_database.loc[index_similar[0], 'ISSN']

        else:
            rating_vhb = None
            issn_vhb = None


        rating_database = J_RATINGS_SCIMAGO
        if publisher in rating_database['Title']:
            rating_sci = rating_database.loc[rating_database['Title'] == publisher, 'H index'].to_numpy()[0]
            issn_sci = rating_database.loc[rating_database['Title'] == publisher, 'Issn'].to_numpy()[0]

        elif index_similar := [i for i, x in enumerate(rating_database['Title']) if is_similar(publisher,x)]:
            rating_sci = rating_database.loc[index_similar[0], 'H index']
            issn_sci = rating_database.loc[index_similar[0], 'Issn']

        else:
            rating_sci = None
            issn_sci = None

        
    return rating_vhb, rating_sci

In [157]:
def combine_publication(row_gs, row_rg):
    
    title = get_not_none_string(str(row_gs['PUB_TITLE']), str(row_rg['PUB_TITLE']))
    
    # title sometimes contains information about type
    if "[BOOK]" in title:
        pub_type = "book"
    else:
        pub_type = row_rg['PUB_TYPE']
    
    title = re.sub("\[[A-Z ]+\]", "", title)
    
    year = max([row_gs['PUB_YEAR'], row_rg['PUB_YEAR']])
    publisher = get_not_none_string(str(row_gs['PUB_PUBLISHER']), str(row_rg['PUB_PUBLISHER']))
    
    authors_gs = [n.strip() for n in ast.literal_eval(row_gs["PUB_AUTHORS"])]
    authors_rg = [n.strip() for n in ast.literal_eval(row_rg["PUB_AUTHORS"])]
    authors = list(set(authors_gs + authors_rg))
    
    data = {
        'AUTHOR_NAME': row_gs["AUTHOR_NAME"],
        'PUB_TITLE': title, 
        'PUB_YEAR': year, 
        'PUB_AUTHORS': authors,  
        'PUB_TYPE': pub_type,
        'PUB_PUBLISHER': publisher,
        'PUB_CITATIONS': row_gs["PUB_CITATIONS"],
        'PUB_ISBN': row_rg['PUB_ISBN'],
        'PUB_DOI': row_rg['PUB_DOI']
       }

    return data

In [158]:
# get all unique names
all_author_names = publications_gs['AUTHOR_NAME'].tolist() + publications_rg['AUTHOR_NAME'].tolist()
all_author_names = set(all_author_names) # make unique

In [159]:
pd_result = pd.DataFrame(columns=['AUTHOR_NAME', 'PUB_TITLE', 'PUB_YEAR', 
                                                      'PUB_AUTHORS', 'PUB_TYPE', 'PUB_PUBLISHER', 
                                                      'PUB_ISBN', 'PUB_DOI', 'PUB_CITATIONS', 'JOU_RATING_VHB', 'JOU_RATING_SCIMAGO'])

In [160]:
if publications_gs.empty:
    pd_result = publications_rg
    
elif publications_rg.empty:
    pd_result = publications_gs
    
else:

    for author_name in all_author_names:

        subset_test_gs = publications_gs[publications_gs['AUTHOR_NAME'] == author_name]
        subset_test_rg = publications_rg[publications_rg['AUTHOR_NAME'] == author_name]

        leftover_rg = subset_test_rg.copy()

        for index_gs, first_row_gs in subset_test_gs.iterrows():
            match_found = False

            # iterate over RG subset
            for index_rg, row in subset_test_rg.iterrows():

                # determining identical based on title and publisher
                if is_similar(row["PUB_TITLE"], first_row_gs["PUB_TITLE"]) and \
                    is_similar(str(row["PUB_PUBLISHER"]), str(first_row_gs["PUB_PUBLISHER"])): 

                    match_found = True

                    combined_row = combine_publication(first_row_gs, row)
                    pd_result = pd_result.append(combined_row, ignore_index=True, sort=False)

                    # remove the found match from the subsets
                    #subset_test_gs = subset_test_gs.drop(index_gs).reset_index(drop=True)
                    if index_rg in leftover_rg.index:
                        leftover_rg = leftover_rg.drop(index_rg)

            if not match_found:
                pd_result = pd_result.append(first_row_gs, ignore_index=True, sort=False)
                #subset_test_gs = subset_test_gs.drop(index_gs).reset_index(drop=True)

            # if there are RG articles left, append them
            if (index_gs + 1 == len(subset_test_gs)) and (len(leftover_rg) > 0):
                pd_result = pd_result.append(leftover_rg, ignore_index=True, sort=False)


pd_result[['JOU_RATING_VHB', 'JOU_RATING_SCIMAGO']] = pd_result.apply(\
        lambda row: get_ratings(row['PUB_PUBLISHER']), axis=1, result_type='expand')

In [161]:
if STORE_PUB:
    pd_result.to_csv(f'../data/{current_year}/publications_{location}.csv', index=False)

In [162]:
pd_result.head(20)

,AUTHOR_NAME,PUB_TITLE,PUB_YEAR,PUB_AUTHORS,PUB_TYPE,PUB_PUBLISHER,PUB_ISBN,PUB_DOI,PUB_CITATIONS,JOU_RATING_VHB,JOU_RATING_SCIMAGO,UPDATED
0,Axel Gerloff,University Offering Work-Integrated Learning D...,2019.0,"['A Gerloff', ' K Reinhard']",NaN,NaN,NaN,NaN,4.0,None,NaN,2022-04-09
1,Axel Gerloff,Internationalizing cooperative education: Impl...,2020.0,"['K Reinhard', ' A Gerloff']",NaN,Int. J. Work. Learn,NaN,NaN,2.0,None,NaN,2022-04-09
2,Axel Gerloff,Stylized facts about stabilization in Central ...,2000.0,['A Gerloff'],NaN,International Advances in Economic Research,NaN,NaN,2.0,C,16.0,2022-04-09
3,Axel Gerloff,[BOOK][B] Wechselkurspolitik in Mittel-und Ost...,2018.0,['A Gerloff '],NaN,NaN,NaN,NaN,5.0,None,NaN,2022-04-09
4,Axel Gerloff,Setting up an online river monitoring station ...,2021.0,"['AS Gerloff', ' JG Wiederhold', ' P Michael…']",NaN,… Monitoring Station of …,NaN,NaN,NaN,None,NaN,2022-04-09
5,Axel Gerloff,Development of an international expansion stra...,2018.0,"['A GERLOFF', ' K REINHARD']",NaN,Refereed Proceedings of the rd …,NaN,NaN,NaN,NaN,771.0,2022-04-09
6,Axel Gerloff,Fiscal Stability in the Eurozone,NaN,['A Gerloff'],NaN,NaN,NaN,NaN,NaN,None,NaN,2022-04-09
7,Dirk Palleduhn,Geschäftsprozessmanagement und integrierte Inf...,2013.0,"['DU Palleduhn', ' H Neuendorf']",NaN,Geschäftsprozessmanagement und …,NaN,NaN,12.0,None,NaN,2022-04-09
8,Frank Elste,Marketing und Werbung in der Medizin: erfolgr...,2018.0,"[Frank Elste, F Elste]",book,nan,978-3-211-83875-4,10.1007/978-3-7091-4063-5,40.0,None,NaN,NaN
9,Frank Elste,Datenanalyse und Konzepte zur Strukturierung v...,1999.0,"[Frank Elste, F Elste]",Article,nan,NaN,NaN,2.0,None,NaN,NaN
